In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile1 = pd.read_pickle("DDD_Train_Decile1.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile1.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile1, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 1')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 3s 1ms/step - loss: 0.5599 - acc: 0.7314 - val_loss: 0.5251 - val_acc: 0.7425
Epoch 36/75
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5542 - acc: 0.7381 - val_loss: 0.5247 - val_acc: 0.7407
Epoch 37/75
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5524 - acc: 0.7460 - val_loss: 0.5241 - val_acc: 0.7407
Epoch 38/75
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5540 - acc: 0.7416 - val_loss: 0.5236 - val_acc: 0.7407
Epoch 39/75
2264/2264 [==============================] - 2s 795us/step - loss: 0.5459 - acc: 0.7367 - val_loss: 0.5236 - val_acc: 0.7425
Epoch 40/75
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5550 - acc: 0.7314 - val_loss: 0.5233 - val_acc: 0.7443
Epoch 41/75
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5470 - acc: 0.7438 - val_loss: 0.5227 - val_acc: 0.7407
Epoch 42/75
2264/2264 [==============================] - 3s 1ms/s

2264/2264 [==============================] - 2s 778us/step - loss: 0.5219 - acc: 0.7434 - val_loss: 0.5078 - val_acc: 0.7584
Epoch 20/75
2264/2264 [==============================] - 2s 765us/step - loss: 0.5192 - acc: 0.7513 - val_loss: 0.5045 - val_acc: 0.7637
Epoch 21/75
2264/2264 [==============================] - 2s 740us/step - loss: 0.5245 - acc: 0.7385 - val_loss: 0.5062 - val_acc: 0.7584
Epoch 22/75
2264/2264 [==============================] - 2s 735us/step - loss: 0.5214 - acc: 0.7527 - val_loss: 0.5048 - val_acc: 0.7672
Epoch 23/75
2264/2264 [==============================] - 2s 767us/step - loss: 0.5220 - acc: 0.7500 - val_loss: 0.5043 - val_acc: 0.7584
Epoch 24/75
2264/2264 [==============================] - 2s 778us/step - loss: 0.5248 - acc: 0.7522 - val_loss: 0.5049 - val_acc: 0.7601
Epoch 25/75
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5208 - acc: 0.7469 - val_loss: 0.5035 - val_acc: 0.7566
Epoch 26/75
2264/2264 [==============================] 

2264/2264 [==============================] - 3s 1ms/step - loss: 0.5886 - acc: 0.6952 - val_loss: 0.5565 - val_acc: 0.7037
Epoch 3/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5769 - acc: 0.7200 - val_loss: 0.5417 - val_acc: 0.7249
Epoch 4/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5607 - acc: 0.7350 - val_loss: 0.5316 - val_acc: 0.7443
Epoch 5/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5474 - acc: 0.7266 - val_loss: 0.5246 - val_acc: 0.7496
Epoch 6/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5546 - acc: 0.7279 - val_loss: 0.5230 - val_acc: 0.7460
Epoch 7/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5522 - acc: 0.7319 - val_loss: 0.5196 - val_acc: 0.7478
Epoch 8/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5425 - acc: 0.7381 - val_loss: 0.5181 - val_acc: 0.7531
Epoch 9/100
2264/2264 [==============================] - 3s 1ms/ste

Epoch 61/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5269 - acc: 0.7443 - val_loss: 0.5088 - val_acc: 0.7619
Epoch 62/100
2264/2264 [==============================] - 2s 961us/step - loss: 0.5323 - acc: 0.7478 - val_loss: 0.5088 - val_acc: 0.7601
Epoch 63/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5257 - acc: 0.7513 - val_loss: 0.5088 - val_acc: 0.7619
Epoch 64/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5263 - acc: 0.7398 - val_loss: 0.5088 - val_acc: 0.7619
Epoch 65/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5236 - acc: 0.7469 - val_loss: 0.5088 - val_acc: 0.7601
Epoch 66/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5326 - acc: 0.7465 - val_loss: 0.5088 - val_acc: 0.7584
Epoch 67/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5271 - acc: 0.7509 - val_loss: 0.5088 - val_acc: 0.7584
Epoch 68/100
2264/2264 [======================

...

2264/2264 [==============================] - 2s 730us/step - loss: 0.5647 - acc: 0.7319 - val_loss: 0.5354 - val_acc: 0.7407
Epoch 26/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5642 - acc: 0.7266 - val_loss: 0.5346 - val_acc: 0.7407
Epoch 27/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5604 - acc: 0.7319 - val_loss: 0.5334 - val_acc: 0.7443
Epoch 28/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5574 - acc: 0.7328 - val_loss: 0.5324 - val_acc: 0.7425
Epoch 29/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5539 - acc: 0.7253 - val_loss: 0.5311 - val_acc: 0.7460
Epoch 30/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5569 - acc: 0.7337 - val_loss: 0.5304 - val_acc: 0.7460
Epoch 31/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5523 - acc: 0.7394 - val_loss: 0.5294 - val_acc: 0.7460
Epoch 32/125
2264/2264 [==============================] - 2

Epoch 85/125
2264/2264 [==============================] - 2s 829us/step - loss: 0.5371 - acc: 0.7469 - val_loss: 0.5159 - val_acc: 0.7619
Epoch 86/125
2264/2264 [==============================] - 2s 846us/step - loss: 0.5418 - acc: 0.7460 - val_loss: 0.5159 - val_acc: 0.7619
Epoch 87/125
2264/2264 [==============================] - 2s 868us/step - loss: 0.5367 - acc: 0.7381 - val_loss: 0.5159 - val_acc: 0.7619
Epoch 88/125
2264/2264 [==============================] - 2s 807us/step - loss: 0.5346 - acc: 0.7381 - val_loss: 0.5158 - val_acc: 0.7619
Epoch 89/125
2264/2264 [==============================] - 2s 813us/step - loss: 0.5425 - acc: 0.7416 - val_loss: 0.5158 - val_acc: 0.7619
Epoch 90/125
2264/2264 [==============================] - 2s 968us/step - loss: 0.5400 - acc: 0.7376 - val_loss: 0.5158 - val_acc: 0.7619
Epoch 91/125
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5405 - acc: 0.7381 - val_loss: 0.5157 - val_acc: 0.7619
Epoch 92/125
2264/2264 [============

2264/2264 [==============================] - 3s 1ms/step - loss: 0.5319 - acc: 0.7509 - val_loss: 0.5125 - val_acc: 0.7584
Epoch 20/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5291 - acc: 0.7531 - val_loss: 0.5130 - val_acc: 0.7619
Epoch 21/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5297 - acc: 0.7460 - val_loss: 0.5099 - val_acc: 0.7549
Epoch 22/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5295 - acc: 0.7496 - val_loss: 0.5091 - val_acc: 0.7601
Epoch 23/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5335 - acc: 0.7447 - val_loss: 0.5123 - val_acc: 0.7637
Epoch 24/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5244 - acc: 0.7465 - val_loss: 0.5083 - val_acc: 0.7601
Epoch 25/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5249 - acc: 0.7473 - val_loss: 0.5070 - val_acc: 0.7601
Epoch 26/100
2264/2264 [==============================] - 3s 

2264/2264 [==============================] - 3s 1ms/step - loss: 0.5053 - acc: 0.7575 - val_loss: 0.5009 - val_acc: 0.7654
Epoch 78/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5134 - acc: 0.7531 - val_loss: 0.5009 - val_acc: 0.7654
Epoch 79/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5186 - acc: 0.7496 - val_loss: 0.5009 - val_acc: 0.7654
Epoch 80/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5113 - acc: 0.7593 - val_loss: 0.5009 - val_acc: 0.7654
Epoch 81/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5119 - acc: 0.7566 - val_loss: 0.5010 - val_acc: 0.7654
Epoch 82/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5080 - acc: 0.7593 - val_loss: 0.5010 - val_acc: 0.7672
Epoch 83/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5171 - acc: 0.7500 - val_loss: 0.5010 - val_acc: 0.7672
Epoch 84/100
2264/2264 [==============================] - 2s 

2264/2264 [==============================] - 3s 1ms/step - loss: 0.5212 - acc: 0.7580 - val_loss: 0.5112 - val_acc: 0.7584
Epoch 37/50
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5194 - acc: 0.7513 - val_loss: 0.5112 - val_acc: 0.7584
Epoch 38/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5250 - acc: 0.7496 - val_loss: 0.5112 - val_acc: 0.7584
Epoch 39/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5245 - acc: 0.7540 - val_loss: 0.5112 - val_acc: 0.7584
Epoch 40/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5210 - acc: 0.7496 - val_loss: 0.5112 - val_acc: 0.7584
Epoch 41/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5280 - acc: 0.7385 - val_loss: 0.5111 - val_acc: 0.7584
Epoch 42/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5220 - acc: 0.7451 - val_loss: 0.5111 - val_acc: 0.7584
Epoch 43/50
2264/2264 [==============================] - 3s 1ms/ste

In [ ]:
0.86434